In [1]:
import pandas as pd
import os
!pip install pykrige

In [2]:
import numpy as np
import pandas as pd
from pykrige.ok import OrdinaryKriging


In [3]:
nlags=20

In [4]:
import pandas as pd
from pykrige.ok import OrdinaryKriging
import numpy as np

def estimate_pollutant(df, date_column, nlags=20):
    # Create a DataFrame with only latitude, longitude, and the specified date column
    df1 = df[['longitude', 'latitude', date_column]]

    # Create a DataFrame with latitude and longitude where the specified date column's values are NaN
    nan_locations_df = df1[df1[date_column].isna()][['longitude', 'latitude']]

    # Create a DataFrame with latitude, longitude, and the specified date column excluding NaN values
    pollutant_df = df1.dropna(subset=[date_column])[['longitude', 'latitude', date_column]]

    # Prepare the data for Ordinary Kriging
    lons = np.array(pollutant_df['longitude'])
    lats = np.array(pollutant_df['latitude'])
    zdata = np.array(pollutant_df[date_column])

    # Perform Ordinary Kriging
    OK = OrdinaryKriging(lons, lats, zdata, variogram_model='linear', verbose=True, enable_plotting=False, nlags=nlags)
    grid_lon = np.array(nan_locations_df["longitude"])
    grid_lat = np.array(nan_locations_df["latitude"])
    z1, ss1 = OK.execute('grid', grid_lon, grid_lat)

    # Fill in NaN values in the specified date column with the interpolated values
    nan_locations_df[date_column] = z1[0]

    # Merge the two DataFrames based on their indices
    merged_df = pd.concat([pollutant_df, nan_locations_df], ignore_index=False)

    # Sort the DataFrame by ascending index
    merged_df = merged_df.sort_index(ascending=True)

    return merged_df

In [ ]:

# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_Hcho/Chosen_state_2023_sep_merge_sentinel5p_Hcho/Delhi_sep_Hcho.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_Hcho/After_krigging_2023_sep_sentinal5p_Hcho/Delhi_2023_sep_O3_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 2.7763356633472323e-08
Nugget: 3.7198027511863187e-08 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...



In [ ]:
result_df

In [ ]:

# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_Hcho/Chosen_state_2023_sep_merge_sentinel5p_Hcho/WB_sep_Hcho.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_Hcho/After_krigging_2023_sep_sentinal5p_Hcho/WB_2023_sep_O3_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 3.951402000181847e-09
Nugget: 4.5246303362898456e-08 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.5998827582284206e-09
Nugget: 1.9373432067197393e-08 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...



In [ ]:
result_df